## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula
from datetime import datetime, timedelta

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst
dsf = tabula.read_pdf('condo.pdf', pages='all')

## Current Week's Values

In [3]:
print('Input Week Starting Date:')
week_starting = input()
print('Input Week Ending Date:')
week_ending = input()
print('Input Current Week Condo Sales (num of units sold):')
current_week_condo_sales = input()
print('Input Average Sale Price:')
average_sale_price = input()
print('Input Average PSF:')
average_price_psf = input()
print('Input Average Days on Market:')
average_days_on_market = input()
print('Input Current Week Condo Sales Total (ex: $XX million):')
current_week_condo_sales_total = input()
print('Input Top Image URL from condo:')
top_condo_image_url = input()

Input Week Starting Date:
Jan. 14th
Input Week Ending Date:
Jan. 20th
Input Current Week Condo Sales (num of units sold):
81
Input Average Sale Price:
871473
Input Average PSF:
543
Input Average Days on Market:
84
Input Current Week Condo Sales Total (ex: $XX million):
70589325
Input Top Image URL from condo:
https://www.condo.com/building/Fendi-Chateau-Ocean-Residences-Surfside-FL-33154-58725728


## Previous Week's Values

In [4]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: $XX million)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
100
Input Previous Week Condo Average Sales Price:
1000933
Input Previous Week Condo Average PSF:
634
Input Previous Week Condo Sales total (ex: $XX million)
100093300


## Clean Data

In [5]:
df = dsf[0]
df.columns = df.iloc[0]
df = df[1:]
df = df.drop(columns=df.columns[0])
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['building_address_city'] = df['building_address_city'].str.replace('^0', '', regex=True)
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
pattern = r'^(?P<building_name>[\d\s]*[A-Za-z\s]+)\s(?P<address>.+)\s(?P<city>[A-Za-z\s]+)$'
df2 = df['geo_address']
df2 = df2.to_frame()
df2[['building_name', 'address', 'city']] = df2['geo_address'].str.extract(pattern, expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [6]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result
df = df.drop(columns='city')

In [7]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [8]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address_x'].apply(geocode)

In [9]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [11]:
# df.at[index#,'col_name']

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=False)

In [12]:
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=False)

In [13]:
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

In [14]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=False)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=False)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [15]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address_x', 'geo_address_y', 'building_name',
       'address', 'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [18]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [19]:
m.save('index.html')

## Data snagger

In [20]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [21]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Fendi Chateau, 9349 Collins Ave 806 | Price $14,250,000 | $3,176 psf | Listing agent: Devin Kay with Douglas Elliman | Buyer's agent: Devin Kay with Douglas Elliman | Days on market: 70
Least Expensive
One Island Place Condo II, 3802 NE 207 St 1401 | Price $1,530,000 | $646 psf | Listing agent: David Deleon with Compass Florida, LLC. | Buyer's agent: Baruh Tchira with Avanti Way Realty LLC | Days on market: 67
Highest Price Per Square Foot
Fendi Chateau, 9349 Collins Ave 806 | Price $14,250,000 | $3,176 psf | Listing agent: Devin Kay with Douglas Elliman | Buyer's agent: Devin Kay with Douglas Elliman | Days on market: 70
Lowest Price Per Square Foot
One Island Place Condo II, 3802 NE 207 St 1401 | Price $1,530,000 | $646 psf | Listing agent: David Deleon with Compass Florida, LLC. | Buyer's agent: Baruh Tchira with Avanti Way Realty LLC | Days on market: 67
Most Days on Market
Residence Du Cap, 2600 Island Blvd PH4 | Price $3,800,000 | $774 psf | Listing agent: Jorge Me

In [22]:
df['days_on_market'].mean()

184.4

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_01202023


# CREATE TEMPLATE 

In [24]:
df['muni_name'] = df['building_address_city'].str.split('\r').str[-1]

In [25]:
muni_set = set(df['muni_name'])

In [26]:
df['building_address_city_clean'] = df['building_address_city'].str.split('\r')

In [27]:
top_sale = df.at[1, 'building_address_city_clean']
second_top_sale = df.at[2,'building_address_city_clean']

In [29]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    

### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [30]:
difference

129460.0

In [31]:
story_string = f'''
\033[1mHED:\033[0m {df.loc[df['int_Sale_Price'].idxmax()]['building_name']} tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Credit condo.com with the image below and the PDF chart in the story
{top_condo_image_url}

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m



Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from condo.com. Condos closed after an average of {average_days_on_market} days on the market.

For the top 10 sales, prices ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}. 

{top_sale[0]}, at {top_sale[1]} in {top_sale[2]} took the top spot with a {df.at[1,'Sale Price']} million closing. {df.at[1,'Agent']} with {df.at[1,'Listing Broker']} had the listing. {df.at[1,'Buyer Agent']} with {df.at[1,'Buyer Broker']} represented the buyer. The sale closed after {df.at[1,'days_on_market']} days on the market at ${df.at[1,'price_per_sqft']} per square foot.

{second_top_sale[0]} in {second_top_sale[2]}, at {second_top_sale[1]}, closed for the second highest amount, {df.at[2,'Sale Price']}, or ${df.at[2,'price_per_sqft']} per square foot, after {df.at[2,'days_on_market']} days on the market. {df.at[2,'Agent']} with {df.at[2,'Listing Broker']} had the listing. {df.at[2,'Buyer Agent']} with {df.at[2,'Buyer Broker']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

# print(story_string)

In [32]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
    
3. Add in a screenshot of the sales chart.
'''

In [38]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address_x', 'geo_address_y', 'building_name',
       'address', 'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR',
       'muni_name', 'building_address_city_clean'],
      dtype='object')

In [40]:
# # Convert each list in 'list_column' to a separate DataFrame and concatenate
# list_df = pd.DataFrame(df['building_address_city_clean'].tolist(), index=df.index)

# # Naming the new columns
# list_df.columns = ['building', 'address', 'city']

# # Concatenate the new DataFrame with the original DataFrame
# df = pd.concat([df, list_df], axis=1).drop('building_address_city_clean', axis=1)

In [43]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address_x', 'geo_address_y', 'building_name',
       'address', 'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR',
       'muni_name', 'building', 'address', 'city'],
      dtype='object')

In [46]:
# print(story_string)

### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

# print(story_checklist)

Most Expensive
Fendi Chateau, address    9349 Collins Ave 806
address    9349 Collins Ave 806
Name: 1, dtype: object | Price $14,250,000 | $3,176 psf | Listing agent: Devin Kay with Douglas Elliman | Buyer's agent: Devin Kay with Douglas Elliman | Days on market: 70
Least Expensive
One Island Place Condo II, address    3802 NE 207 St 1401
address    3802 NE 207 St 1401
Name: 10, dtype: object | Price $1,530,000 | $646 psf | Listing agent: David Deleon with Compass Florida, LLC. | Buyer's agent: Baruh Tchira with Avanti Way Realty LLC | Days on market: 67
Highest Price Per Square Foot
Fendi Chateau, address    9349 Collins Ave 806
address    9349 Collins Ave 806
Name: 1, dtype: object | Price $14,250,000 | $3,176 psf | Listing agent: Devin Kay with Douglas Elliman | Buyer's agent: Devin Kay with Douglas Elliman | Days on market: 70
Lowest Price Per Square Foot
One Island Place Condo II, address    3802 NE 207 St 1401
address    3802 NE 207 St 1401
Name: 10, dtype: object | Price $1,530,